## Setup

In [1]:
# imports
import os
os.environ['KERAS_BACKEND'] = 'torch'

import numpy as np
import keras
import pandas as pd
from mne_bids import BIDSPath, read_raw_bids
from typing import List
import xarray as xr
from pathlib import Path
from scipy.signal import resample, butter, sosfiltfilt, iirnotch, tf2sos
from IPython.display import clear_output
from tqdm import tqdm

## Preprocessing OTKA

In [45]:
def process_input(path: str,
                  subject: str,
                  task: str,
                  channels: List[str],
                  resampling_frq=200,
                  notchfilter_frq=50,
                  filter_bounds=[0.3, 75],
                  verbose=False,
                  microvolts=False,
                  pretrain=False):

    # open data
    print(f'>>>>>>>>preprocessing {subject}-{task}')
    bids_path = BIDSPath(subject=subject,
                         session='01',
                         task=task,
                         root=path
                         )
    raw = read_raw_bids(bids_path, extra_params={'preload': True}, verbose=verbose)
    raw.set_montage('standard_1020')

    # pick eeg channels
    print('picking eeg channels...')
    # ['T3', 'T4', 'T5', 'T6'] channels that are only in the 10-20 system
    # replaced with their equivalent name in the 10-10 system [T7, T8, P7, P8]
    raw.pick(channels, verbose=verbose)

    # interpolate bad channels if there is any
    print('interpolating bad channels...')
    raw.interpolate_bads(verbose=verbose)

    # resampling
    if resample is not None:
        raw.resample(resampling_frq, verbose=verbose)

    raw.filter(l_freq=filter_bounds[0], h_freq=filter_bounds[1], verbose=verbose)
    raw.notch_filter((notchfilter_frq), verbose=verbose)
    eeg_array = raw.get_data().T
    points, chs = eeg_array.shape
    if microvolts:
        eeg_array = eeg_array * 10**6
    
    if pretrain:  # pretrain mode follows their pretraining's preprocessing steps
        a = points % (30 * 200)
        eeg_array = eeg_array[60 * 200:-(a+60 * 200), :]
        eeg_array = eeg_array.reshape(-1, 30, 200, chs)
    
    else:
        eeg_array = eeg_array[:200*416, :]  # trim time dim so that it is dividable by 200 (time is just a few timepoints longer than this)
        eeg_array = eeg_array.reshape(-1, 2, 200, chs)
        eeg_array = eeg_array.reshape(-1, 2, 200, chs)

    eeg_array = eeg_array.transpose(0, 3, 1, 2)

    return eeg_array

In [38]:
eeg_path = '/Volumes/Extreme_SSD/PhD/OTKA_study1/EEG_data/BIDS/'
behvioral_path = '../EEGModalNet/data/OTKA/PLB_HYP_data_MASTER.csv'

behavioral = pd.read_csv(behvioral_path).dropna(subset='bids_id')
behavioral['bids_id'] = behavioral['bids_id'].apply(lambda x: f'{int(x):02}')
bid_id = behavioral['bids_id'].values
behavioral.set_index('bids_id', inplace=True)
gender = behavioral['gender']
hypnotizability = behavioral['hypnotizability_total']

In [ ]:
PRETRAIN = False
subjects = [f'{i:02}' for i in range(1, 52)]
tasks = ['experience1', 'experience2', 'experience3', 'experience4']
if PRETRAIN:
    channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz']
else:
    channels = ['O1', 'O2', 'P1', 'P2', 'C1', 'C2', 'F1', 'F2']

def load_trial(path, subj, task):
    eeg_data = process_input(
        path=path,
        subject=subj,
        task=task,
        channels=channels,
        verbose=0,
        pretrain=PRETRAIN
    )
    return eeg_data

def process_subject_trials(path, subj, tasks):
    trial_data = np.vstack(np.array([load_trial(path, subj, task) for task in tasks]))
    clear_output()
    return trial_data

all_eeg_data = [process_subject_trials(eeg_path, subj, tasks) for subj in subjects]

In [ ]:
if PRETRAIN:  # since appending this data will create inhomogenous dimension we will remove it in the test mode
    # Two sessions are missing in the Last participants' data so we process and append it separately
    subj = '52'
    tasks = ['experience1', 'experience4']
    sub_52_data = process_subject_trials(eeg_path, subj, tasks)

    all_eeg_data.append(sub_52_data)

    # update subjects ids accordingly 
    subjects = [f'{i:02}' for i in range(1, 53)]

In [ ]:
if PRETRAIN:
    segments_to_be_excluded = {}

    for sub in range(len(all_eeg_data)):
        sample_key = []
        for i, sample, in enumerate(all_eeg_data[sub]):
            if np.max(np.abs(sample)) > 100:
                sample_key.append(i)
        segments_to_be_excluded[f'sub_{sub}'] = sample_key

    # exclude bad epochs
    filtered_x = []

    for sub in range(len(all_eeg_data)):
        excluded_segments = segments_to_be_excluded[f'sub_{sub}']
        filtered_sub = np.delete(all_eeg_data[sub], excluded_segments, axis=0)  # Remove excluded segments
        filtered_x.append(filtered_sub)

    # Convert the filtered list back to a numpy array
    x = np.array(filtered_x, dtype=object)

    all_segments = []
    subject_ids = []
    epoch_ids = []
    gender_of_epoch = []
    hypnotizablity_of_epoch = []

    for subj_idx, subj_array in zip(subjects, x):
        for epoch_idx, segment in enumerate(subj_array):
            all_segments.append(segment)
            subject_ids.append(subj_idx)
            gender_of_epoch.append(gender[subj_idx])
            hypnotizablity_of_epoch.append(hypnotizability[subj_idx])
            epoch_ids.append(epoch_idx)

    # Step 2: Convert list to array
    all_segments = np.stack(all_segments)  # shape (total_epochs, 30, 128)

    subject_epoch = [f'{sub_id}_epoch-{epo_id}' for sub_id, epo_id in zip(subject_ids, epoch_ids)]

    data = xr.DataArray(
        all_segments,
        dims=("subject_epoch", "channel", "segment", "time"),
        coords={
            "subject_epoch": subject_epoch,
            "channel": channels,
            "segment": np.arange(all_segments.shape[2]),
            "time": np.arange(all_segments.shape[3]),
        },
        attrs={'gender': gender_of_epoch,
            'hypnotizablity': hypnotizablity_of_epoch},
        name="eeg"
    )

    # save
    # data.to_netcdf('data/OTKA_preprocessed_for_Cbramod.nc5', engine='h5netcdf')

In [ ]:
if not PRETRAIN:
    all_segments = np.array(all_eeg_data)

    # removing last participant's data
    hypnotizability_ = hypnotizability.iloc[:-1].values
    gender_ = gender.iloc[:-1].values

    data = xr.DataArray(
        all_segments,
        dims=("subject", "epoch", "channel", "segment", "time"),
        coords={
            "subject": subjects,
            "epoch": np.arange(all_segments.shape[1]),
            "channel": channels,
            "segment": np.arange(all_segments.shape[3]),
            "time": np.arange(all_segments.shape[4]),
        },
        attrs={'gender': gender_,
            'hypnotizablity': hypnotizability_},
        name="eeg"
    )

    # data.to_netcdf('data/OTKA_preprocessed_for_Cbramod_downstream.nc5', engine='h5netcdf')

## Feature extraction

In [6]:
ds = xr.open_dataarray('data/OTKA_preprocessed_for_Cbramod.nc5')

In [ ]:
from argparse import Namespace
import torch 
from torch import nn
from CBraMod.models.cbramod import CBraMod
from einops.layers.torch import Rearrange

DEFAULT_PARAMS = Namespace(**{
    "foundation_dir": "pretrained_weights/pretrained_weights.pth",
    "features_file_path": "data/CBraMod_features_<DOWNSTREAM_TASK>.pt",
    "num_of_classes": 2,
    "device": 'cpu',

    "data_dir": "data/LEMON/",
    "channels": ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz'],
    "downstream_task": "gender",
    "segment_size": 30,  # TODO
    "batch_size": 1024,
    "bandpass_filter": 0.3,
    "n_channels": 19,
    "n_segments": 2,  # TODO ?

})

DEFAULT_PARAMS.features_file_path = DEFAULT_PARAMS.features_file_path.replace(
    "<DOWNSTREAM_TASK>", DEFAULT_PARAMS.downstream_task.lower())

params = DEFAULT_PARAMS

x = torch.tensor(ds.to_numpy()).float()

In [ ]:
# feature extraction
batch_size = 50  # Adjust batch size based on available memory
num_batches = len(x) // batch_size + (1 if len(x) % batch_size != 0 else 0)

backbone = CBraMod(
    in_dim=200, out_dim=200, d_model=200,
    dim_feedforward=800, seq_len=30,
    n_layer=12, nhead=8
)

backbone.load_state_dict(
    torch.load(params.foundation_dir,
                map_location=torch.device(params.device), weights_only=True))

backbone.eval()
backbone.proj_out = nn.Identity()

for batch_idx in tqdm(range(num_batches), leave=True):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(x))
    batch = x[start_idx:end_idx]
    bz, ch_num, seq_len, patch_size = batch.shape
    features = backbone(batch)
    features = Rearrange('b c s p -> b (c s p)')(features).contiguous()
    # store batch features
    torch.save(features.detach(),
               f"data/OTKA_extracted_features/CBraMod_features_{batch_idx:02}.pt")

100%|██████████| 15/15 [01:45<00:00,  7.01s/it]


In [14]:
all_features = torch.zeros((x.shape[0], features.shape[1]))
batch_size = 50
for i, path_dir in enumerate(sorted(Path('data/OTKA_extracted_features/').glob('*.pt'))):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(x))
    feature = torch.load(path_dir, weights_only=True)
    all_features[start_idx:end_idx] = feature

In [16]:
# open all the batch data, and concat them
subject_ids = [i.split('_')[0] for i in ds.subject_epoch.values]
gender = ds.gender
gender = [0 if i=='Female' else 1 for i in gender]

In [18]:
# store all features
torch.save({'features': all_features,
            'gender': gender,
            'subject_ids': subject_ids}, params.features_file_path)
print("features saved to", params.features_file_path)

features saved to data/LEMON/CBraMod_features_gender.pt


## Classification

In [2]:
import torch
DO_DOWN_SAMPLE = False
data = torch.load('data/OTKA_extracted_features/CBraMod_features_gender.pt', weights_only=True)
X_e = np.array(data['features'])
y = np.array(data['gender'])
groups = np.array(data['subject_ids'])

if DO_DOWN_SAMPLE:
    y_1_idx = np.where(y==1)[0]
    # we know that there is less male (class 1) than female, so we down sample based on the lenght of data in class 1
    y_0_idx = np.where(y==0)[0][:len(y_1_idx)]
    all_idx = np.append(y_0_idx, y_1_idx)

    # now we downsample features, subject ids and gender based on these indeces
    X_e = X_e[all_idx]
    y = y[all_idx]
    groups = groups[all_idx]

/var/folders/pn/qy0qc5tx7t7_97xjmy6pvksc0000gn/T/ipykernel_18660/2302037879.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  X_e = np.array(data['features'])


In [ ]:
# SPLIT
from sklearn.model_selection import ShuffleSplit, GroupShuffleSplit
DO_GROUPED_SHUFFLE = True

if DO_GROUPED_SHUFFLE:
    group_shuffle = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    train_idx, val_idx = next(group_shuffle.split(X_e, y, groups=groups))
else:
    random_shuffle = ShuffleSplit(n_splits=1, test_size=0.3, random_state=1)
    train_idx, val_idx = next(random_shuffle.split(X_e, y))

y.mean(), y[train_idx].mean(), y[val_idx].mean()

(np.float64(0.5),
 np.float64(0.5059288537549407),
 np.float64(0.48623853211009177))

In [ ]:
from keras import regularizers, layers

cls_model = keras.models.Sequential([   
    layers.Dense(64, activation='gelu'),
    layers.Dense(1, activation='sigmoid')
])

cls_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy'])

history = cls_model.fit(
    X_e[train_idx], y[train_idx],
    epochs=500,
    batch_size=128,
    validation_data=(X_e[val_idx], y[val_idx]),
    shuffle=True)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 713ms/step - accuracy: 0.4882 - loss: 3.2566 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 688ms/step - accuracy: 0.4960 - loss: 8.1228 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 627ms/step - accuracy: 0.5013 - loss: 8.0388 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step - accuracy: 0.4908 - loss: 8.2067 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 654ms/step - accuracy: 0.4830 - loss: 8.3326 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - accuracy: 0.4960 - loss: 8.1228 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step - accuracy: 0.4908 - loss: 8.2067 - val_accuracy: 0.5138 - val_loss: 7.8372
Epoch 8/500
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 445ms/step - accuracy: 0.5391 - loss: 7.4294

KeyboardInterrupt: 

## preprocessing Lemon

In [ ]:
# open lemon data

# channels: ['T3', 'T4', 'T5', 'T6'] channels that are only in the 10-20 system replaced with their equivalent name in the 10-10 system [T7, T8, P7, P8].
channels = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3',
            'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2']  # FIXME: odering might be different in the data they inputed to their model

n_subjects = 202
lemon = xr.open_dataset('data/LEMON/eeg_eo_ec.nc5')

x = lemon['eye_closed'].sel(subject=lemon.subject[:n_subjects], channel=channels).to_numpy() * 10**6  # Volts to microvolts

# FIXME resampling to 200Hz
# n_samples = int((x.shape[-1] / sampling_rate) * downsample_frq)
# x = resample(x, num=n_samples, axis=-1)

# bandpass filter
sos = butter(4, 0.3, btype='highpass', fs=128, output='sos')
x = sosfiltfilt(sos, x, axis=-1)

# notchfilter
fs = 128
f0 = 50
Q = 30
w0 = f0 / (fs / 2)  # Normalized Frequency
b, a = iirnotch(w0, Q)
sos = tf2sos(b, a)
x = sosfiltfilt(sos, x, axis=-1)

subs, chs, points = x.shape

# epoching
x = x.reshape(subs, -1, chs, 30, 128)  # subjects, segments, channels, 30 seconds, sampling rate

In [ ]:
segments_to_be_excluded = {}

for sub in range(x.shape[0]):
    sample_key = []
    for i, sample, in enumerate(x[sub]):
        if np.max(np.abs(sample)) > 100:
            sample_key.append(i)
    segments_to_be_excluded[f'sub_{sub}'] = sample_key

# exclude bad epochs
filtered_x = []

for sub in range(x.shape[0]):
    excluded_segments = segments_to_be_excluded[f'sub_{sub}']
    filtered_sub = np.delete(x[sub], excluded_segments, axis=0)  # Remove excluded segments
    filtered_x.append(filtered_sub)

# Convert the filtered list back to a numpy array
x = np.array(filtered_x, dtype=object)

In [ ]:
demog = pd.read_csv('data/LEMON/Demographics.csv')
gender = demog['Gender_ 1=female_2=male'].values
age = demog['Age']

In [ ]:
all_segments = []
subject_ids = []
epoch_ids = []
gender_of_epoch = []
age_of_epoch = []

for subj_idx, subj_array in enumerate(x):
    for epoch_idx, segment in enumerate(subj_array):
        all_segments.append(segment)
        subject_ids.append(lemon.subject.values.tolist()[subj_idx])
        gender_of_epoch.append(int(gender[subj_idx]))
        age_of_epoch.append(age[subj_idx])
        epoch_ids.append(epoch_idx)

# Step 2: Convert list to array
all_segments = np.stack(all_segments)  # shape (total_epochs, 30, 128)

subject_epoch = [f'{sub_id}_epoch-{epo_id}' for sub_id, epo_id in zip(subject_ids, epoch_ids)]

data = xr.DataArray(
    all_segments,
    dims=("subject_epoch", "channel", "segment", "time"),
    coords={
        "subject_epoch": subject_epoch,
        "channel": channels,
        "segment": np.arange(all_segments.shape[2]),
        "time": np.arange(all_segments.shape[3]),
    },
    attrs={'gender': gender_of_epoch,
           'age': age_of_epoch},
    name="eeg"
)

In [ ]:
# data.to_netcdf('data/LEMON/lemon_preprocessed_for_cbramod.nc5', engine='h5netcdf')